# Creating Type Classes and Instances

## Outline

* Revisiting Type Classes
* The `Eq` type class
	* Defining the `Eq` type class
	* Defining an instance for the `Eq` type class
	* Improving our `Eq` type class (minimal complete definition)
	* Defining an instance for a parameterize type.
* The `Ord` type class
	* Exploring `Ord` type class (Subclassing)
* Deriving
* Complete example


In this lesson, first, we'll do a quick recap of type classes, learn how to create them from scratch, and how to use them.

## Revisiting Type Classes

* Type classes are interfaces that define some behavior. (For example, comparing for equality.)

* If a type is an instance of a type class, it supports the behaviors of that type class. Meaning that we can use the functions the type class defines.

* A type class can have multiple instances, and a type can belong to multiple type classes.

## The `Eq` type class

### Defining the `Eq` type class

Remember that the `Eq` type class is all about equality. The types that are instances of the `Eq` type class can say if two values of its type are equal or different by using the `==` (equal) and `/=` (not equal) functions.

But how do we define the `Eq` class? We can do it ike this:

```haskell
class Eq a where
  (==) :: a -> a -> Bool
  (/=) :: a -> a -> Bool
```

* `class ... where` means that we're defining a new type class.
* `Eq` is how type class will be called.
* The `a` inside `Eq a` is the type variable that holds the place to the type that will be made an instance of this type class.
* Then, we define the functions of the type class (`==` and `/=`).

But wait.. the functions aren't implemented!

That's because, you can do it, but there's no **need** to actually implement the functions in the type class. **Only the type delcarations are required!**

Now that we have a type class, let's use it! 

### Defining an instance for the `Eq` type class

Let's define a type for the payment methods a customer can use:

In [ ]:
data PaymentMethod = Cash | Card | CC -- CC stands for Cryptocurrency

Now, lets make `PaymentMethod` an instance of `Eq`, so we can check its values for equality.

In [ ]:
instance Eq PaymentMethod where
  -- Implementation of ==
  Cash == Cash = True
  Card == Card = True
  CC == CC = True
  _ == _ = False
  
  -- Implementation of /=
  Cash /= Cash = False
  Card /= Card = False
  CC /= CC = False
  _ /= _ = True

* `instance ... where` means that we're making some type an instance of a type class
* `Eq PaymentMethod` means that `PaymentMethod` is the type that will be come an instance of `Eq` type class. 
* Then, we implement the functions of the type class (`==` and `/=`) for our specific type.

As you can see, because now we're creating an instance for a type, we replace the type variable (`a`) that we had in the type class definition for that specific type (`PaymentMethod`).

And that's it! That's how you define a type class and how you make a type an instance of it! Now, `PaymentMethod` can freely use the `Eq` behaviours (`==` and `/=`):

In [ ]:
Card == Cash
CC /= Card

But here's the thing. We're on our way to become epic Haskell developers, and epic Haskell developers can do better than that.

### Improving our `Eq` type class

With our `Eq` type class defined like that, each time we need to make a type an instance of `Eq`, we have to define both `==` and `/=`. But we know that we can also define functions inside the type class definition, and we know that for each pair of values, `==` and `/=` give use the opposite `Bool` value. 

So, taking into account the relationship between `==` and `/=`, we can redefine `Eq` type class like this:

```haskell
class  Eq a  where
    (==), (/=)  :: a -> a -> Bool
    x /= y      = not (x == y)
    x == y      = not (x /= y)
```

**Which is how `Eq` is actually defined in the standard library!**

By defining `==` and `/=` as the oposite of the other one (mutual recursion), now Haskell can infer the behavior of one from the other. And each time we make a type an instance of the `Eq` type class, we just need to overwrite (implement) one of them! Like this:

In [ ]:
instance Eq PaymentMethod where
  Cash == Cash = True
  Card == Card = True
  CC == CC = True
  _ == _ = False

And the type can use both `==` and `/=` no questions asked!

This is called **minimal complete definition**. 

You can take advantage of this by checking what's the minimal complete definition of a type class using `:i <type class>` and implementing only those behaviors. If you do `:i Eq`, you'll get:

```haskell
type Eq :: * -> Constraint -- Eq takes a concrete type and  returns a Constraint
class Eq a where
  (==) :: a -> a -> Bool
  (/=) :: a -> a -> Bool
  {-# MINIMAL (==) | (/=) #-}
  	-- Defined in ‘GHC.Classes’

-- ... and a bunch of instances.
```

In this line:

```haskell
{-# MINIMAL (==) | (/=) #-}
```

It says that to have the *minimal complete implementation* of the class, you have to implement either `==` OR `/=`.

In [ ]:
-- Check by yourself!
:i Eq

Let's use our newly acquired knowledge to create an instance of a parameterized type!

### Defining an instance of a parameterized type

First, we define our new type:

In [ ]:
data Box a = Empty | Has a

But we can't do something like this:

In [ ]:
instance Eq Box where
-- ...

We'd get an Error! Because we're trying to pass a type constructor (`* -> *`) instead of a concrete type (`*`)!

How do we know we need a concrete type? Because if we take a look at the type type declarations:

```haskell
type Eq :: * -> Constraint
class Eq a where
(==) :: a -> a -> Bool
(/=) :: a -> a -> Bool
```

The type declarations clearly use `a` as a concrete type, and `type Eq :: * -> Constraint` clearly states that `Eq` takes a concrete type.

It makes sense. We can't have a function of type `a -> Box`, but we can have one of type `a -> Box a`.

So, we could satisfy this by using:

```haskell
class Eq (Box Int) where
-- ...

class Eq (Box String) where
-- ...

class Eq (Box PaymentMethod) where
-- ...

--- etc
```

A class for every type. Hmm, we already went through this, and we solved it with type variables. This time isn't different!:

```haskell
instance Eq (Box a) where
-- ...
```

This way, all of the type of form `Box a` (like `Box String` or `Box Int`) will be an instance of `Eq`.

As a general rule, when makig istances for type constructors, you have to provide enough parameters so that you end up obtaining a concrete type.

And, if we decide that two boxes that contain the same element are equal, we can define the behaviours as:

In [ ]:
instance Eq (Box a) where
  Has x == Has y = x == y
  Empty == Empty = True
  _ == _ = False

But there's a HUGE mistake with this instance! Did you spotted it? Did you even run the cell?

Exactly! We're using `==` between two values (`x` and `y`) of type `a` without making sure that the `a` type itself is an instance of `Eq`! Let's correct that:

In [ ]:
instance (Eq a) => Eq (Box a) where
  Has x == Has y = x == y
  Empty == Empty = True
  _ == _ = False

Now, this is what I call an instance!

Let's take a quick look at the `Ord` type class

## The `Ord` type class

### Exploring the `Ord` type class (Subclassing)

Run this code ant take a look at the info:

In [ ]:
:i Ord

As you can see, the information (`:i`) of the `Ord` type class looks something like this:

```haskell
type Ord :: * -> Constraint         -- Ord takes a concreate type and returns a Constraint
class Eq a => Ord a where           -- That "Eq a =>" is new!! 🤔 (explained below)
  compare :: a -> a -> Ordering     -- Don't know what the "Ordering" type is, but I can check it out using ":i Ordering"
  (<) :: a -> a -> Bool             -- A bunch of function definitions
  (<=) :: a -> a -> Bool
  (>) :: a -> a -> Bool
  (>=) :: a -> a -> Bool
  max :: a -> a -> a
  min :: a -> a -> a
  {-# MINIMAL compare | (<=) #-}    -- We can implement the type class defining only "compare" or "<=".
```

Everything checks out. Except for that `Eq a =>`.

This (`Eq a =>`) it's called **subclassing**, and it means exactly what you'd imagine:

**To make a type an instance of `Ord`, first we have to make it an instance or `Eq`! Meaning that `Eq` is a prerequesite for `Ord`**

It makes sense, because how can you know how to order the values of a type if you can't know if two values are equal!

Luckly, we already created an instance of `Box` for the `Eq` type class, so we can create one for the `Ord` class without any problems. We just pick one of the functions needed for the minimal complete implementation, and code it!:

In [ ]:
instance (Ord a) => Ord (Box a) where
  Has x <= Has y = x <= y
  _ <= _ = False

This is all well and good, but what if I need a type to behave like `Ord`, `Num`, and `Show`? Most common cases are implemented the same, and that's a lot of repetitive work. Luckily,—thanks to some really smart guys with PhDs—most of the time, we don't need to actually do it! We can just derive the instance!

## Deriving

Derived instances are an automatic way of making a type an instance a type class.

They are possible only for classes known to the compiler: those defined in either the Prelude or a standard library (`Eq`, `Ord`, `Enum`, ...). When in doubt, try, and the compiler will yell at you and give you useful information if it can't do it.

To use this feature, just add `deriving (...)` (with the names of all the type classes that you want to derive) at the end of your type declaration.

Each type class has its own set of rules for deriving instances. Let's see a few of the most common:

#### `deriving (Show)`

This one is the simplest. It gives you access to functions that convert your type's values into Strings:

In [ ]:
data Box a = Empty | Has a deriving (Show)

box = Has 5
result = "There's a box that " ++ show box -- Thanks to deriving (Show), you can use the "show" function.

In [ ]:
data Person = Person
  { name :: String,
    company :: String,
    experience :: Float,
    position :: String
  }
  deriving (Show)

person = Person "Rick" "IOG" 7.5 "Scientist" 
person -- Thanks to deriving (Show), you can print the type's values on the console.

### `deriving (Eq)`

Deriving can go wrong (see: https://www.youtube.com/watch?v=1txgSlcpQmo)

### `deriving (Ord)`

The value generated from the constructor that was defined first is smaller. And if values are made from the same constructor and the constructor doesn't have fields, they are equal. For example:

In [ ]:
data PaymentMethod = Cash | Card | CC deriving (Ord)

Cash > Card
Card < CC
CC `compare` Cash

If values are made from the same constructor and the constructor does have fields, the fields are compared (In this case, the types of the fields also have to be instances of `Ord`):

In [ ]:
data Box a = Empty | Has a deriving (Show, Ord)

Empty `compare` Empty
Has 5 `compare` Has 6
Has "Hello" `compare` Has "Hi"

### Full example (Leave as homework?)

Imagine we want to create a type that represents the position in a company:

In [ ]:
data Position = Intern | Junior | Senior | Manager | Chief deriving (Show, Read, Eq, Ord, Enum, Bounded)

## Recap